In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
import pickle
import string
from multiprocessing import Pool
from multiprocessing import Process, Manager
import datetime
import xml.etree.ElementTree as ET
import json
import re
import fnmatch
import random
import shutil

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from tqdm import tqdm
from pycococreatortools import pycococreatortools

In [2]:
!unzip pycococreatortools.zip

Archive:  pycococreatortools.zip
   creating: pycococreatortools/.ipynb_checkpoints/
  inflating: pycococreatortools/.ipynb_checkpoints/pycococreatortools-checkpoint.py  
 extracting: pycococreatortools/__init__.py  
   creating: pycococreatortools/__pycache__/
  inflating: pycococreatortools/__pycache__/__init__.cpython-37.pyc  
  inflating: pycococreatortools/__pycache__/pycococreatortools.cpython-37.pyc  
  inflating: pycococreatortools/pycococreatortools.py  


In [6]:
#!unzip Training.zip
#!unzip Val.zip
!unzip Test.zip


Archive:  Test.zip
  inflating: Test/Aerialimg0170.jpg  
  inflating: Test/Aerialimg0171.jpg  
  inflating: Test/Aerialimg0172.jpg  
  inflating: Test/Aerialimg0173.jpg  
  inflating: Test/Aerialimg0174.jpg  
  inflating: Test/Aerialimg0175.jpg  
  inflating: Test/Aerialimg0176.jpg  
  inflating: Test/Aerialimg0177.jpg  
  inflating: Test/Aerialimg0178.jpg  
  inflating: Test/Aerialimg0179.jpg  
  inflating: Test/Aerialimg0180.jpg  
  inflating: Test/Aerialimg0181.jpg  
  inflating: Test/Aerialimg0182.jpg  
  inflating: Test/Aerialimg0183.jpg  
  inflating: Test/Aerialimg0184.jpg  
  inflating: Test/Aerialimg0185.jpg  
  inflating: Test/Aerialimg0186.jpg  
  inflating: Test/Aerialimg0187.jpg  
  inflating: Test/Aerialimg0188.jpg  
  inflating: Test/Aerialimg0189.jpg  
  inflating: Test/Aerialimg0190.jpg  
  inflating: Test/Aerialimg0191.jpg  
  inflating: Test/Aerialimg0192.jpg  
  inflating: Test/Aerialimg0193.jpg  
  inflating: Test/Aerialimg0194.jpg  
  inflating: Test/Aerialimg0195

In [7]:
Root_Path = '/content'

In [8]:
DIR_XML = Root_Path + '/data/xmls'
DIR_SHAPES = Root_Path + '/data/shapes/'
DIR_IMG = Root_Path + '/data/images/'
DIR_ANN = Root_Path + '/data/annotations/'

In [9]:
len(os.listdir(DIR_XML)) 

200

In [10]:
fnames_images = sorted(os.listdir(DIR_IMG))

## Creating Imageid Dict

In [12]:
dict_imageid_to_fname = dict()

In [13]:
len(fnames_images)

200

In [14]:
# i=132
for i,fname in enumerate(fnames_images):
    dict_imageid_to_fname[i] = fname

In [15]:
 dict_imageid_to_fname

{0: 'Aerialimg0170.jpg',
 1: 'Aerialimg0171.jpg',
 2: 'Aerialimg0172.jpg',
 3: 'Aerialimg0173.jpg',
 4: 'Aerialimg0174.jpg',
 5: 'Aerialimg0175.jpg',
 6: 'Aerialimg0176.jpg',
 7: 'Aerialimg0177.jpg',
 8: 'Aerialimg0178.jpg',
 9: 'Aerialimg0179.jpg',
 10: 'Aerialimg0180.jpg',
 11: 'Aerialimg0181.jpg',
 12: 'Aerialimg0182.jpg',
 13: 'Aerialimg0183.jpg',
 14: 'Aerialimg0184.jpg',
 15: 'Aerialimg0185.jpg',
 16: 'Aerialimg0186.jpg',
 17: 'Aerialimg0187.jpg',
 18: 'Aerialimg0188.jpg',
 19: 'Aerialimg0189.jpg',
 20: 'Aerialimg0190.jpg',
 21: 'Aerialimg0191.jpg',
 22: 'Aerialimg0192.jpg',
 23: 'Aerialimg0193.jpg',
 24: 'Aerialimg0194.jpg',
 25: 'Aerialimg0195.jpg',
 26: 'Aerialimg0196.jpg',
 27: 'Aerialimg0197.jpg',
 28: 'Aerialimg0198.jpg',
 29: 'Aerialimg0199.jpg',
 30: 'Plane001.jpg',
 31: 'Plane002.jpg',
 32: 'Plane003.jpg',
 33: 'Plane004.jpg',
 34: 'Plane005.jpg',
 35: 'Plane006.jpg',
 36: 'Plane007.jpg',
 37: 'Plane008.jpg',
 38: 'Plane009.jpg',
 39: 'Plane010.jpg',
 40: 'Plane011.jpg',

In [16]:
os.getcwd()

'/content'

In [18]:
with open('/content/data/pickle_files/dict_imageid_to_fname1.pkl', 'wb') as file:  
    pickle.dump(dict_imageid_to_fname, file)

In [19]:
with open('/content/data/pickle_files/dict_imageid_to_fname1.pkl', 'rb') as f:
    dict_imageid_to_fname = pickle.load(f)

In [20]:
list(dict_imageid_to_fname.items())[0]

(0, 'Aerialimg0170.jpg')

In [21]:
dict_fname_to_imageid = dict()

In [22]:
for imageid,fname in dict_imageid_to_fname.items():
    dict_fname_to_imageid[fname] = imageid

In [23]:
with open('/content/data/pickle_files/dict_imageid_to_fname1.pkl', 'wb') as f:
    pickle.dump(dict_fname_to_imageid, f)
    

In [24]:
with open('/content/data/pickle_files/dict_imageid_to_fname1.pkl', 'rb') as f:
    dict_fname_to_imageid = pickle.load(f)

In [25]:
list(dict_fname_to_imageid.items())[4]

('Aerialimg0174.jpg', 4)

## Creating binary masks

In [26]:
def get_xml_filename_using_imageid(imageid):
    fname = dict_imageid_to_fname[imageid]
    fname = fname[:-4]
    return fname + '.xml'

In [27]:
def get_xml_filepath_using_imageid(imageid):
    fname = get_xml_filename_using_imageid(imageid)
    path = os.path.join(DIR_XML, fname)
    return path

In [28]:
def get_image_filepath_using_imageid(imageid):
    fname = str(imageid) + '.jpg'
    path = os.path.join(DIR_SHAPES, fname)
    return path

In [29]:
get_xml_filename_using_imageid(1)

'Aerialimg0171.xml'

In [30]:
get_xml_filepath_using_imageid(1)

'/content/data/xmls/Aerialimg0171.xml'

In [31]:
fname = 'Aerialimg0171.xml'
dict_fname_to_imageid[fname[:-4]+'.jpg']

1

In [32]:
path_xml = get_xml_filepath_using_imageid(1)
path_xml

'/content/data/xmls/Aerialimg0171.xml'

In [33]:
tree = ET.parse(path_xml)
root = ET.parse(path_xml).getroot()
root.findall('./object/[name="MilitaryJeep"]')

[<Element 'object' at 0x7fb4d282a638>]

In [34]:
os.listdir(DIR_XML)[1]

'Plane008.xml'

In [36]:
len(os.listdir(DIR_ANN))

0

In [37]:
invalid = []
for fname in tqdm(os.listdir(DIR_XML)):
    try:
        imageid = dict_fname_to_imageid[fname[:-4]+'.jpg']
        path_image = os.path.join(DIR_IMG, fname[:-4]+'.jpg')
        print(path_image)
        path_xml = get_xml_filepath_using_imageid(imageid)
        print(path_xml)
        tree = ET.parse(path_xml)
        root = tree.getroot()    
        obj_1 = root.findall("./object/[name='Warplane']/bndbox")
        obj_2 = root.findall("./object/[name='Plane']/bndbox")
        obj_3 = root.findall("./object/[name='Tank']/bndbox")
        obj_4 = root.findall("./object/[name='Warship']/bndbox")
        obj_5 = root.findall("./object/[name='Ship']/bndbox")
        obj_6 = root.findall("./object/[name='Person']/bndbox")
        obj_7 = root.findall("./object/[name='MilitaryTruck']/bndbox")
        obj_8 = root.findall("./object/[name='MilitaryJeep']/bndbox")
        image = plt.imread(path_image)
        shape_image = image.shape
        annotation_id = 0        
            
        if obj_1:
            for obj in obj_1:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Warplane", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        if obj_2:
            for obj in obj_2:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Plane", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        if obj_3:
            for obj in obj_3:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Tank", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
              
        if obj_4:
            for obj in obj_4:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Warship", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        if obj_5:
            for obj in obj_5:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Ship", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        if obj_6:
            for obj in obj_6:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "Person", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        if obj_7:
            for obj in obj_7:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "MilitaryTruck", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
        
        if obj_8:
            for obj in obj_8:
                x1, y1, x2, y2 = [int(i.text) for i in obj]
                mask = np.zeros((shape_image[0], shape_image[1], 3))
                mask_plus = np.ones((y2-y1, x2-x1,3))
                mask[y1:y2,x1:x2,:] = mask_plus
                path_bin = "_".join([str(imageid), "MilitaryJeep", str(annotation_id)]) + '.jpg'
                path_bin = os.path.join(DIR_ANN, path_bin)
                plt.imsave(path_bin, mask)
                annotation_id+=1
            
        path_src = os.path.join(DIR_IMG, fname[:-4]+'.jpg')
        path_dest = os.path.join(DIR_SHAPES, str(imageid) + '.jpg')
        shutil.copy(path_src, path_dest)
    except Exception as e:
#         print(e)
        invalid.append([path_image, path_xml])

  0%|          | 1/200 [00:00<00:20,  9.72it/s]

/content/data/images/Warship016.jpg
/content/data/xmls/Warship016.xml
/content/data/images/Plane008.jpg
/content/data/xmls/Plane008.xml
/content/data/images/Plane001.jpg
/content/data/xmls/Plane001.xml
/content/data/images/Warplane022.jpg
/content/data/xmls/Warplane022.xml
/content/data/images/Tank808.jpg
/content/data/xmls/Tank808.xml


  6%|▌         | 11/200 [00:00<00:12, 15.70it/s]

/content/data/images/Ship015.jpg
/content/data/xmls/Ship015.xml
/content/data/images/Plane016.jpg
/content/data/xmls/Plane016.xml
/content/data/images/Ship018.jpg
/content/data/xmls/Ship018.xml
/content/data/images/Warship021.jpg
/content/data/xmls/Warship021.xml
/content/data/images/Plane020.jpg
/content/data/xmls/Plane020.xml
/content/data/images/Tank555.jpg
/content/data/xmls/Tank555.xml
/content/data/images/Warplane036.jpg
/content/data/xmls/Warplane036.xml
/content/data/images/Warship005.jpg
/content/data/xmls/Warship005.xml
/content/data/images/Plane004.jpg
/content/data/xmls/Plane004.xml
/content/data/images/Aerialimg0195.jpg
/content/data/xmls/Aerialimg0195.xml


  9%|▉         | 18/200 [00:00<00:09, 19.49it/s]

...

/content/data/images/Warship013.jpg
/content/data/xmls/Warship013.xml
/content/data/images/Warplane005.jpg
/content/data/xmls/Warplane005.xml
/content/data/images/Ship005.jpg
/content/data/xmls/Ship005.xml
/content/data/images/Warplane019.jpg
/content/data/xmls/Warplane019.xml


 98%|█████████▊| 196/200 [00:07<00:00, 16.04it/s]

/content/data/images/Ship006.jpg
/content/data/xmls/Ship006.xml
/content/data/images/Ship021.jpg
/content/data/xmls/Ship021.xml
/content/data/images/Warplane017.jpg
/content/data/xmls/Warplane017.xml
/content/data/images/Warplane010.jpg
/content/data/xmls/Warplane010.xml
/content/data/images/Plane022.jpg
/content/data/xmls/Plane022.xml
/content/data/images/Warplane020.jpg
/content/data/xmls/Warplane020.xml
/content/data/images/Ship022.jpg
/content/data/xmls/Ship022.xml
/content/data/images/Aerialimg0196.jpg
/content/data/xmls/Aerialimg0196.xml


100%|██████████| 200/200 [00:08<00:00, 24.50it/s]

/content/data/images/Plane003.jpg
/content/data/xmls/Plane003.xml
/content/data/images/Ship030.jpg
/content/data/xmls/Ship030.xml


In [38]:
len(os.listdir(DIR_SHAPES))

200

In [39]:
len(invalid)

0

In [40]:
imgs = sorted(os.listdir(DIR_SHAPES))

In [41]:
random.shuffle(imgs)

In [42]:
INFO = {
    "description": "ObjectDetection",
    "url": "example.com",
    "version": "0.1.0",
    "year": 2019,
    "contributor": "GB",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "Some License",
        "url": "http://somewebsite.com/"
    }
]
####
# Warplane
# Plane
# Tank
# Warship
# Ship
# Person
# MilitaryTruck
# MilitaryJeep

CATEGORIES = [
    {
        'id': 1,
        'name': 'Warplane',
        'supercategory': 'objdet',
    },
    {
        'id': 2,
        'name': 'Plane',
        'supercategory': 'objdet',
    },
    {
        'id': 3,
        'name': 'Tank',
        'supercategory': 'objdet',
    },
     {
        'id': 4,
        'name': 'Warship',
        'supercategory': 'objdet',
    },
     {
        'id': 5,
        'name': 'Ship',
        'supercategory': 'objdet',
    },
    {
        'id': 6,
        'name': 'Person',
        'supercategory': 'objdet',
    },
     {
        'id': 7,
        'name': 'MilitaryTruck',
        'supercategory': 'objdet',
    },
    {
        'id': 8,
        'name': 'MilitaryJeep',
        'supercategory': 'objdet',
    },
    
]

In [43]:
INFO

{'contributor': 'GB',
 'date_created': '2020-08-09 09:35:24.987598',
 'description': 'ObjectDetection',
 'url': 'example.com',
 'version': '0.1.0',
 'year': 2019}

In [44]:
IMAGE_DIR =DIR_SHAPES
ANNOTATION_DIR = DIR_ANN
ROOT_DIR= Root_Path

In [45]:
def filter_for_jpeg(root, files):
    file_types = ['*.jpeg', '*.jpg']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    return files



In [46]:
def filter_for_annotations(root, files, image_filename):
    image_basename = os.path.basename(image_filename)
    files = [os.path.join(root,f) for f in files if f.split('_')[0] == image_basename[:-4]]
    return files

In [47]:
len(os.listdir(IMAGE_DIR))

200

## Coco Json File Creation

In [52]:
def main():

    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }

    image_id = 1
    segmentation_id = 1
    
    # filter for jpeg images
    for root, _, files in os.walk(IMAGE_DIR):
        image_files = filter_for_jpeg(root, files)

        # go through each image
        for image_filename in image_files:
            image = Image.open(image_filename)
            image_info = pycococreatortools.create_image_info(
                image_id, os.path.basename(image_filename), image.size)
            coco_output["images"].append(image_info)

            # filter for associated png annotations
            for root, _, files in os.walk(ANNOTATION_DIR):
                annotation_files = filter_for_annotations(root, files, image_filename)

                # go through each associated annotation
                for annotation_filename in annotation_files:
                    print(annotation_filename)
                    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]
                    category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
                    binary_mask = np.asarray(Image.open(annotation_filename)
                        .convert('1')).astype(np.uint8)
                    
                    annotation_info = pycococreatortools.create_annotation_info(
                        segmentation_id, image_id, category_info, binary_mask,
                        image.size, tolerance=2)

                    if annotation_info is not None:
                        coco_output["annotations"].append(annotation_info)

                    segmentation_id = segmentation_id + 1

            image_id = image_id + 1

    with open('{}/military_coco_creation_test.json'.format(ROOT_DIR), 'w') as output_json_file:
    #with open('{}/military_coco_creation_val.json'.format(ROOT_DIR), 'w') as output_json_file:
    #with open('{}/military_coco_creation_train.json'.format(ROOT_DIR), 'w') as output_json_file:  
        json.dump(coco_output, output_json_file)


In [53]:
main()

/content/data/annotations/199_Warship_0.jpg
/content/data/annotations/74_Ship_0.jpg
/content/data/annotations/132_Warplane_0.jpg
/content/data/annotations/106_Tank_0.jpg
/content/data/annotations/77_Ship_0.jpg
/content/data/annotations/172_Warship_0.jpg
/content/data/annotations/91_Tank_0.jpg
/content/data/annotations/91_Person_1.jpg
/content/data/annotations/141_Warplane_0.jpg
/content/data/annotations/65_Ship_0.jpg
/content/data/annotations/160_Warplane_4.jpg
/content/data/annotations/160_Warplane_3.jpg
/content/data/annotations/160_Warplane_1.jpg
/content/data/annotations/160_Warplane_7.jpg
/content/data/annotations/160_Warplane_2.jpg
/content/data/annotations/160_Warplane_8.jpg
/content/data/annotations/160_Warplane_0.jpg
/content/data/annotations/160_Warplane_5.jpg
/content/data/annotations/160_Warplane_6.jpg
/content/data/annotations/67_Ship_0.jpg
/content/data/annotations/144_Warplane_0.jpg
/content/data/annotations/54_Plane_0.jpg
/content/data/annotations/158_Warplane_0.jpg
/co